In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import SGD


In [2]:
with open("history.txt",'r') as historyFile:
        lines = historyFile.readlines()
        gamehistory=[]
        for x in lines:
            x=x[1:]
            x=x[:-2]
            gamehistory.append(x)
        print(gamehistory)

        

['1, 1, 5, 4, 3, 0, 8, 7', '-1, 4, 8, 1, 6, 0, 2, 3, 5', '-1, 4, 3, 5, 0, 7, 1, 6, 2', '-1, 7, 0, 5, 8, 6, 4', '-1, 6, 5, 0, 8, 1, 4, 7, 2', '1, 5, 4, 7, 3, 2, 0, 6, 1, 8', '1, 4, 0, 8, 3, 7, 5, 6', '1, 4, 3, 8, 5, 1, 7, 2, 0, 6', '1, 4, 6, 5, 2, 3', '-1, 5, 4, 0, 1, 2, 7']


In [3]:
inputGame=[]
output=[]

for games in gamehistory:
    if(games[0]=='-'):
        inputGame.append(-1)
        output.append(games[1:])
    
print(inputGame)
print(output)

[-1, -1, -1, -1, -1]
['1, 4, 8, 1, 6, 0, 2, 3, 5', '1, 4, 3, 5, 0, 7, 1, 6, 2', '1, 7, 0, 5, 8, 6, 4', '1, 6, 5, 0, 8, 1, 4, 7, 2', '1, 5, 4, 0, 1, 2, 7']


In [4]:
def createData(dataset):
        inputGame = []
        output = []
        for data in dataset:
            inputGame.append(data[1])
            output.append(data[0])

        X = np.array(inputGame).reshape((-1, len(inputGame)))
        y = to_categorical(output, num_classes=3)
        # Train and test data split
        boundary = int(0.8 * len(X))
        x_train = X[:boundary]
        x_test = X[boundary:]
        y_train = y[:boundary]
        y_test = y[boundary:]


In [5]:
model = keras.Sequential()
model.add(layers.Dense(18, input_dim=9,kernel_initializer='normal', activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(9, kernel_initializer='normal',activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1,kernel_initializer='normal'))

learning_rate = 0.001
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum,nesterov=False)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 18)                180       
_________________________________________________________________
dropout (Dropout)            (None, 18)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 171       
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10        
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [6]:
def legal_moves_generator(current_board_state):
    """Function that returns the set of all possible legal moves and resulting board states, 
    for a given input board state and player

    Args:
    current_board_state: The current board state


    Returns:
    legal_moves_dict: A dictionary of a list of possible next coordinate-resulting board state pairs
    The resulting board state is flattened to 1 d array

    """
    current_board_state = np.array(current_board_state).reshape(3,3)
    legal_moves_dict={}
    for i in range(current_board_state.shape[0]):
        for j in range(current_board_state.shape[1]):
            if current_board_state[i,j]==0:
                board_state_copy=current_board_state.copy()
                legal_moves_dict[(i,j)]=board_state_copy.flatten()
    return legal_moves_dict

In [7]:
4, 5, 0, 3, 6, 1, 8
    selected_move=max(tracker, key=tracker.get)

    new_board_state=legal_moves_dict[selected_move]
    score=tracker[selected_move]

    return selected_move,new_board_state,score


In [11]:
import os
import random
import copy

#X's are 1, O's are -1, empty is 0
#check if 3 in a row has been made
def checkWin(board):
    win = 0
    winningLines = [[0, 1, 2],[3, 4, 5],[6, 7, 8],[0, 3, 6],[1, 4, 7],[2, 5, 8],[0, 4, 8],[2, 4, 6]]
    for x in winningLines:
        if( board[x[0]]==1 and  board[x[1]]==1 and  board[x[2]]==1):
            win = 1
        elif(board[x[0]]==-1 and  board[x[1]]==-1 and  board[x[2]]==-1):
            win = -1
    return win
#find which parts of the array are available
def findLegalMoves(board):
    legalMoves=[]
    for i in range(len(board)):
        if board[i] == 0:
            legalMoves.append(i)
    return legalMoves

#why didn't I use a dictionary
def createBoard(board):
    physicalBoard=[]
    for x in board:
        if x == -1:
            physicalBoard.append('O')
        elif x == 1:
            physicalBoard.append('X')
        elif x == 0:
            physicalBoard.append(" ")

    return physicalBoard
#display board
def printBoard(board):
    physicalBoard = createBoard(board)

    print(physicalBoard[0]+"|"+physicalBoard[1]+"|"+physicalBoard[2])
    print("-+-+-")
    print(physicalBoard[3]+"|"+physicalBoard[4]+"|"+physicalBoard[5])
    print("-+-+-")
    print(physicalBoard[6]+"|"+physicalBoard[7]+"|"+physicalBoard[8]+"\n")
def printHelpBoard():
    print("\nChoose a place to put your piece using the following guide:")
    print("0"+"|"+"1"+"|"+"2")
    print("-+-+-")
    print("3"+"|"+"4"+"|"+"5")
    print("-+-+-")
    print("6"+"|"+"7"+"|"+"8\n")



#initialize empty board
board = [0,0,0, 0,0,0 ,0,0,0]
#begin counting pieces played
piecesPlayed = 0
gameOver = False

#load model
model =  keras.models.load_model("tictactoe1000_model.h5")

while(piecesPlayed<9 and gameOver == False):
    #display empty board
    printBoard(board)
    #find legal moves
    legalMoves = findLegalMoves(board)
    playerInput=-1
    #State which player moves
    if(piecesPlayed%2==0):
        print("Player X's Turn")
        while (playerInput<0 or playerInput>8 or playerInput not in legalMoves):
            printHelpBoard()
            playerInput = int(input("Where would you like to place your piece?"))
            if playerInput not in legalMoves:
                print("Please choose a legal move.")
            elif( playerInput <0 or playerInput >8):
                print("Please choose a move in the game.")
            else:
                board[playerInput]=1
    else:
        print("Player O's Turn")
        print("Generating random move for O")
        playerInput,board,score=move_selector(model,board)
        playerInput=playerInput[0]*3+playerInput[1]
        board[playerInput]=-1

    piecesPlayed = piecesPlayed + 1
    win = checkWin(board)
    if (win == 1):
        print("Player X Wins")
        gameOver = True
        printBoard(board)
    elif(win == -1):
        print("Player O Wins")
        gameOver = True
        printBoard(board)
    elif(piecesPlayed == 9):
        print("Game is a draw.")
        gameOver = True
        printBoard(board)


 | | 
-+-+-
 | | 
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 0


X| | 
-+-+-
 | | 
-+-+-
 | | 

Player O's Turn
Generating random move for O
X|O| 
-+-+-
 | | 
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 3


X|O| 
-+-+-
X| | 
-+-+-
 | | 

Player O's Turn
Generating random move for O
X|O|O
-+-+-
X| | 
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 6


Player X Wins
X|O|O
-+-+-
X| | 
-+-+-
X| | 

